In [1]:
using RigidBodyDynamics, RigidBodyDynamics.Contact, RigidBodyDynamics.OdeIntegrators
using MomentumBasedControl
using MixedIntegerExperiments
using StaticArrays
using RigidBodyTreeInspector, DrakeVisualizer

In [2]:
DRAKE_DISTRO = "$(ENV["HOME"])/code/drake-distro"
examples_path = "$DRAKE_DISTRO/drake/examples"
urdf = joinpath(examples_path, "Valkyrie/urdf/urdf/valkyrie_A_sim_drake_one_neck_dof_wide_ankle_rom.urdf")

"/home/twan/code/drake-distro/drake/examples/Valkyrie/urdf/urdf/valkyrie_A_sim_drake_one_neck_dof_wide_ankle_rom.urdf"

In [3]:
mechanism, feet, hands, pelvis, hippitches, knees, anklepitches, floatingjoint = create_valkyrie(urdf);

In [4]:
# Create visualizer
DrakeVisualizer.any_open_windows() || (DrakeVisualizer.new_window(); sleep(1))
vis = Visualizer(parse_urdf(urdf, mechanism; package_path = [examples_path]), :valkyrie);
visualize_environment(mechanism, vis)

In [5]:
remove_fixed_tree_joints!(mechanism)
const state = MechanismState(Float64, mechanism)

MechanismState{Float64, Float64, Float64}(…)

In [6]:
# Initial setup
zero!(state)
kneebend = 1.
for side in instances(Side)
    set_configuration!(state, knees[side], [kneebend])
    set_configuration!(state, hippitches[side], [-kneebend / 2])
    set_configuration!(state, anklepitches[side], [-kneebend / 2])
end
set_configuration!(state, floatingjoint, [1; 0; 0; 0; 0; 0; 1.08]) # TODO
settransform!(vis, state);
com0 = center_of_mass(state)

Point3D in "world": [0.0200832,-0.000967307,1.05451]

In [7]:
# const τ = Vector{Float64}(num_velocities(mechanism))
# const result = DynamicsResult(Float64, mechanism)
# function damped_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
#     τ[:] = velocity(state)
#     scale!(τ, -1.0)
#     dynamics!(result, state, τ)
#     copy!(vd, result.v̇)
#     copy!(sd, result.ṡ)
# end

In [8]:
const controller = MomentumBasedController{Float64}(mechanism);

In [9]:
for body in bodies(mechanism)
    for point in contact_points(body)
        set_contact_regularization!(controller, point, 1e-1)
    end
end
set_joint_accel_regularization!(controller, 0.05)
comgains = PDGains(100., 20.)

MomentumBasedControl.PDGains{Float64}(100.0,20.0)

In [10]:
const result = DynamicsResult(Float64, mechanism)
function controlled_dynamics!(vd::AbstractArray, sd::AbstractArray, t, state)
    # TODO: extract out: highlevelcontrol
    # Foot accelerations
    T = eltype(controller)
    root = root_body(mechanism)
    for foot in values(feet)
        accel = zero(SpatialAcceleration{T}, default_frame(foot), default_frame(root), default_frame(foot))
        set_desired_accel!(controller, root, foot, accel)
        for point in contact_points(foot)
            set_contact_active!(controller, point, true)
        end
    end
    
    # Centroidal momentum control
    com = center_of_mass(state)
    centroidal = centroidal_frame(controller)
    world_to_centroidal = Transform3D(com.frame, centroidal, -com.v)
    h = transform(momentum(state), world_to_centroidal)
    comerror = FreeVector3D(transform(com0, world_to_centroidal))
    comderror = -FreeVector3D(h.frame, h.linear)
    ldot_desired = comgains.k * comerror + comgains.d * comderror
    hdot_desired = Wrench(ldot_desired, zero(ldot_desired))
    set_desired_momentum_rate!(controller, zero(Wrench{T}, centroidal), eye(SMatrix{6, 6, T}))
    
    τ = control(controller, t, state)
    dynamics!(result, state, τ)
    copy!(vd, result.v̇)
    copy!(sd, result.ṡ)
end

controlled_dynamics! (generic function with 1 method)

In [11]:
# Simulation
const dyn! = controlled_dynamics!
const integrator = MuntheKaasIntegrator(dyn!, runge_kutta_4(Float64), DrakeVisualizerSink(vis));

In [15]:
controlled_dynamics!(result.v̇, result.ṡ, 0., state);

comerror = FreeVector3D in "centroidal": [0.00432516,-0.000400745,0.00997365]


In [18]:
result.v̇

36-element Array{Float64,1}:
  -0.0467156 
   9.94829   
  -0.22172   
  -0.0957702 
   0.00903177
   0.538354  
   0.221596  
 -11.8115    
  10.6999    
 -12.2291    
   4.12651   
  -3.5867    
   0.0533784 
   ⋮         
   0.27634   
   0.907208  
  -2.32004   
   2.14363   
   0.0594306 
   0.0398695 
   0.358509  
  -0.218917  
  -1.53318   
   1.62993   
  -0.281196  
  -0.024353  

In [12]:
integrate(integrator, state, 3., 5e-4, maxRealtimeRate = Inf)

comerror = FreeVector3D in "centroidal": [0.0,0.0,0.0]
comerror = FreeVector3D in "centroidal": [0.0,0.0,0.0]
comerror = FreeVector3D in "centroidal": [-4.78371e-12,2.55934e-11,6.13114e-7]
comerror = FreeVector3D in "centroidal": [-2.81539e-13,-1.84805e-11,1.22625e-6]
comerror = FreeVector3D in "centroidal": [1.54547e-12,-1.13368e-11,1.22625e-6]
comerror = FreeVector3D in "centroidal": [-7.73569e-13,-1.07195e-11,2.45247e-6]
comerror = FreeVector3D in "centroidal": [8.40376e-13,-1.59305e-12,3.06562e-6]
comerror = FreeVector3D in "centroidal": [2.70033e-12,-3.20425e-12,4.90505e-6]
comerror = FreeVector3D in "centroidal": [1.21838e-12,-8.97516e-12,4.905e-6]
comerror = FreeVector3D in "centroidal": [-2.45328e-12,-5.85195e-12,7.35739e-6]
comerror = FreeVector3D in "centroidal": [2.73909e-12,-1.11721e-11,7.97067e-6]
comerror = FreeVector3D in "centroidal": [4.38043e-12,-1.35154e-11,1.10364e-5]
comerror = FreeVector3D in "centroidal": [1.37873e-12,-9.47292e-12,1.10363e-5]
comerror = FreeVecto

LoadError: InterruptException: